In [14]:
import time
import numpy as np
import torch
import warnings
import torch.nn as nn
from typing import Tuple
import numpy as np
from robustbench.data import get_preprocessing, load_clean_dataset
from robustbench.model_zoo.enums import BenchmarkDataset, ThreatModel
from robustbench.utils import clean_accuracy
from pathlib import Path
from typing import Callable, Optional, Sequence, Tuple, Union
from autoattack import AutoAttack

In [15]:
def benchmark(
    model: Union[nn.Module, Sequence[nn.Module]],
    n_examples: int = 10000,
    dataset: Union[str, BenchmarkDataset] = BenchmarkDataset.cifar_10,
    threat_model: Union[str, ThreatModel] = ThreatModel.Linf,
    model_name: Optional[str] = None,
    data_dir: str = "./data",
    device: Optional[Union[torch.device, Sequence[torch.device]]] = None,
    batch_size: int = 32,
    eps: Optional[float] = None,
    log_path: Optional[str] = None,

    version='custom', #'rand', # 'rand' for models using stochasticity

    preprocessing: Optional[Union[str, Callable]] = None,
    aa_state_path: Optional[Path] = None,
    image_path: Optional[str] = None,

    ) -> Tuple[float, float]:
    if isinstance(model, Sequence) or isinstance(device, Sequence):
        # Multiple models evaluation in parallel not yet implemented
        raise NotImplementedError

    try:
        if model.training:
            warnings.warn(Warning("The given model is *not* in eval mode."))
    except AttributeError:
        warnings.warn(
            Warning(
                "It is not possible to asses if the model is in eval mode"))

    dataset_: BenchmarkDataset = BenchmarkDataset(dataset)
    threat_model_: ThreatModel = ThreatModel(threat_model)
    model = model.cuda()

    prepr = get_preprocessing(dataset_, threat_model_, model_name,
                            preprocessing)

    clean_x_test, clean_y_test = load_clean_dataset(dataset_, n_examples,
                                                    data_dir, prepr)

    clean_accs = []
    for i in range(10):
        accuracy = clean_accuracy(model,
                                clean_x_test,
                                clean_y_test,
                                batch_size=batch_size,
                                device=device)
        clean_accs.append(accuracy)
    print(f'With {len(clean_x_test)} clean examples, test for 10 times, clean accuracy: {np.mean(clean_accs):.2%}±{np.std(clean_accs):.2%}')

    do_auto_attack = True
    if do_auto_attack:
        # AutoAttack
        adversary = AutoAttack(
                            model,  # at default, this will call aggreegated model
                            norm=threat_model_.value,
                            eps=eps,
                            version=version,
                            device=device,
                            log_path=log_path,
                            attacks_to_run=['apgd-ce', 'apgd-t'] if version == "custom" else [], #Stan's addition
                            )
        x_adv = adversary.run_standard_evaluation(
                                                clean_x_test,
                                                clean_y_test,
                                                bs=batch_size,
                                                state_path=aa_state_path)
        
        #calculate the robust accuracy for 10 times, then report the average accuracy and std
        accs = []
        for i in range(10):
            adv_accuracy = clean_accuracy(
                                        model,
                                        x_adv,
                                        clean_y_test,
                                        batch_size=batch_size,
                                        device=device)
            accs.append(adv_accuracy)
        print(f'After auto attack, with {len(clean_x_test)} clean examples,  test for 10 times, robust accuracy: {np.mean(accs):.2%}±{np.std(accs):.2%}')
        print("===================================")
    
    else:
        print("Skip AutoAttack")
    # note: load adversarial images generated by our attack
    saved_adv_images = np.load(f"{image_path}")
    saved_adv_images = torch.tensor(saved_adv_images).cuda()
    accs = []
    for i in range(10):
        adv_accuracy = clean_accuracy(
                                    model,
                                    saved_adv_images[:len(clean_y_test)],
                                    clean_y_test,
                                    batch_size=batch_size,
                                    device=device)
        accs.append(adv_accuracy)
    print("===================================")
    print(f'After our attack, with {len(clean_x_test)} clean examples,  test for 10 times, robust accuracy: {np.mean(accs):.2%}±{np.std(accs):.2%}')

In [16]:
from helpers.utils import setup_seed
setup_seed(1)

#! test it on cifar100
wrapped_combined_model = torch.load('/data/projects/ensem_adv/saved_ckpts/full_model.pth')
data_dir = "/local/home/jiezha/data/"


time_start = time.time()
# benchmark(
#         wrapped_combined_model.eval(),
#         dataset="cifar100",
#         threat_model='Linf',
#         device=torch.device("cuda"),
#         eps=8/255,
#         n_examples=100,  # test on 100 examples
#         version='rand',
#         batch_size=32,
#         data_dir=data_dir,
#         image_path="saved/saved_adv_images_final_cifar100.npy"
#     )
# time_end = time.time()
# print(f"Time cost:  {(time_end-time_start)/60} mins")

In [17]:
#! test it on cifar10

wrapped_combined_model = torch.load('/data/projects/ensem_adv/saved_ckpts/full_model_cifar10.pth')

time_start = time.time()
benchmark(
        wrapped_combined_model.eval(),
        dataset="cifar10",
        threat_model='Linf',
        device=torch.device("cuda"),
        eps=8/255,
        n_examples=100,  # test on 100 examples
        version='rand',
        batch_size=32,
        data_dir=data_dir,
        image_path="saved/saved_adv_images_final_cifar10.npy"
    )
time_end = time.time()
print(f"Time cost:  {(time_end-time_start)/60} mins")

Files already downloaded and verified
With 100 clean examples, test for 10 times, clean accuracy: 86.60%±2.01%
setting parameters for rand version
using rand version including apgd-ce, apgd-dlr.
initial accuracy: 83.00%
apgd-ce - 1/3 - 17 out of 32 successfully perturbed
apgd-ce - 2/3 - 13 out of 32 successfully perturbed
apgd-ce - 3/3 - 7 out of 19 successfully perturbed
robust accuracy after APGD-CE: 46.00% (total time 2705.6 s)
apgd-dlr - 1/2 - 4 out of 32 successfully perturbed
apgd-dlr - 2/2 - 1 out of 14 successfully perturbed
robust accuracy after APGD-DLR: 41.00% (total time 4256.6 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 41.00%
After auto attack, with 100 clean examples,  test for 10 times, robust accuracy: 61.80%±2.32%


FileNotFoundError: [Errno 2] No such file or directory: 'saved/saved_adv_images_final_cifar10.npy'